<a href="https://colab.research.google.com/github/AUT-Student/NN-HW6/blob/main/NN_HW6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Library

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import glob
from copy import deepcopy
from IPython.display import display

import tensorflow as tf
import tensorflow.keras as keras
from keras.layers import Input, Dense, LSTM, GRU, BatchNormalization, Dropout
from keras.models import Sequential
from keras.callbacks import EarlyStopping

# Dataset

## Load

In [ ]:
!gdown 1SLrmOfW66WQc-zrfo-gX-g2dDA4pAw1F

Downloading...
From: https://drive.google.com/uc?id=1SLrmOfW66WQc-zrfo-gX-g2dDA4pAw1F
To: /content/NN_HW6_Dataset.zip
100% 1.80M/1.80M [00:00<00:00, 126MB/s]


In [ ]:
!unzip /content/NN_HW6_Dataset.zip

Archive:  /content/NN_HW6_Dataset.zip
  inflating: IRX6XAFF0005.csv        
  inflating: IRX6XALS0002.csv        
  inflating: IRX6XS300003.csv        
  inflating: IRX6XSLC0000.csv        
  inflating: IRX6XSNT0009.csv        
  inflating: IRX6XTAL0001.csv        
  inflating: IRX6XTDP0004.csv        
  inflating: IRX6XTPI0009.csv        
  inflating: IRX6XTPI0009[2022-05-23-10-18-57].csv  
  inflating: IRX6XTPI0025.csv        
  inflating: IRX6XWAI0001.csv        
  inflating: IRX6XWTH0001.csv        
  inflating: IRXWXEXR0007.csv        
  inflating: IRXWXEXR0023.csv        
  inflating: IRXWXOCI0001.csv        
  inflating: IRXWXOCI0027.csv        
  inflating: IRXYXTPI0009.csv        
  inflating: IRXYXTPI0025.csv        
  inflating: IRXZXAGR0009.csv        
  inflating: IRXZXBNK0006.csv        
  inflating: IRXZXCMI0000.csv        
  inflating: IRXZXCML0005.csv        
  inflating: IRXZXCNS0007.csv        
  inflating: IRXZXENG0009.csv        
  inflating: IRXZXEXR0004.csv      

## Merge

In [ ]:
pathes = glob.glob("/content/*.csv")

merged_dataset = None

for path in pathes:
  sub_dataset = pd.read_csv(path, encoding="UTF-16", delimiter=",")
  sub_dataset = sub_dataset[sub_dataset["<DTYYYYMMDD>"] >= 20210522]

  if len(sub_dataset)>=238:
    index_name = sub_dataset["<COl12>"].values[0]
    print(path)
    print(index_name)
    print(sub_dataset["<TICKER>"].values[0])
    sub_dataset = sub_dataset[["<DTYYYYMMDD>",	"<OPEN>",	"<HIGH>", "<LOW>",
                               "<CLOSE>",	"<VOL>",	"<OPENINT>",	"<OPENINT>.1",
                               "<OPENINT>.2"	]]

    sub_dataset = sub_dataset.add_prefix(prefix=index_name + " ")

    sub_dataset = sub_dataset.rename({f"{index_name} <DTYYYYMMDD>": "<DTYYYYMMDD>"}, axis=1)

    display(sub_dataset)

    if merged_dataset is None:
      merged_dataset = deepcopy(sub_dataset)
    else:
      merged_dataset = merged_dataset.merge(sub_dataset, on="<DTYYYYMMDD>", how="inner")

    print("##############\n\n")

/content/IRXZXOCI0008.csv
OTC Overall Index
شاخص_كل_فرابورس6


,<DTYYYYMMDD>,OTC Overall Index <OPEN>,OTC Overall Index <HIGH>,OTC Overall Index <LOW>,OTC Overall Index <CLOSE>,OTC Overall Index <VOL>,OTC Overall Index <OPENINT>,OTC Overall Index <OPENINT>.1,OTC Overall Index <OPENINT>.2
1944,20210522,17810.8,17816.8,17676.4,17676.4,443477753,3.223291e+12,90258,1.183554e+10
1945,20210523,17642.9,17642.9,17503.9,17503.9,598258196,3.181944e+12,98344,1.169097e+10
1946,20210524,17499.8,17504.5,17476.5,17476.5,529694599,2.683259e+12,115169,1.165787e+10
1947,20210525,17474.0,17552.8,17552.8,17552.8,605556635,3.236629e+12,205382,1.170879e+10
1948,20210526,17553.4,17582.5,17582.3,17582.3,528637682,2.898542e+12,233866,1.172845e+10
...,...,...,...,...,...,...,...,...,...
2177,20220516,21553.8,21620.7,21620.7,21620.7,2465155499,1.561998e+13,345395,1.190409e+10
2178,20220517,21644.3,21695.6,21689.7,21689.7,2629661250,1.531904e+13,303832,1.200937e+10
2179,20220518,21620.7,21671.6,21643.8,21643.8,1843016966,1.139007e+13,262424,1.197906e+10
2180,20220521,21612.6,21612.6,21539.4,21539.4,1422075970,8.965528e+12,182173,1.192066e+10


##############


/content/IRX6XSNT0009.csv
Industy Index
شاخص_صنعت6


,<DTYYYYMMDD>,Industy Index <OPEN>,Industy Index <HIGH>,Industy Index <LOW>,Industy Index <CLOSE>,Industy Index <VOL>,Industy Index <OPENINT>,Industy Index <OPENINT>.1,Industy Index <OPENINT>.2
3007,20210522,992706.0,992706.0,974675.0,974675.0,2138516996,1.113298e+13,132814,3.637847e+10
3008,20210523,968432.0,968432.0,956210.0,956210.0,1960930556,1.185582e+13,155363,3.565174e+10
3009,20210524,953029.0,953453.0,950072.0,950072.0,1892545187,1.245752e+13,195946,3.540903e+10
3010,20210525,953288.0,961846.0,961846.0,961846.0,2220851084,1.441892e+13,263828,3.580313e+10
3011,20210526,963015.0,966806.0,965719.0,965719.0,2210639177,1.271506e+13,309912,3.595344e+10
...,...,...,...,...,...,...,...,...,...
3241,20220516,1469370.0,1474700.0,1474700.0,1474700.0,8469576514,4.493093e+13,685740,5.472715e+10
3242,20220517,1483100.0,1498830.0,1497690.0,1497690.0,8366404232,4.467476e+13,753123,5.558027e+10
3243,20220518,1494950.0,1494950.0,1487150.0,1487150.0,7188312476,3.688831e+13,858950,5.518893e+10
3244,20220521,1478060.0,1478060.0,1469790.0,1469790.0,6502579033,3.478897e+13,511516,5.452978e+10


##############


/content/IRX6XAFF0005.csv
Free Float Index
شاخص_آزاد_شناور6


...

##############


/content/IRXYXTPI0025.csv
TotalEqualWeightPr
شاخص_قيمت(هم_وزن6)


,<DTYYYYMMDD>,TotalEqualWeightPr <OPEN>,TotalEqualWeightPr <HIGH>,TotalEqualWeightPr <LOW>,TotalEqualWeightPr <CLOSE>,TotalEqualWeightPr <VOL>,TotalEqualWeightPr <OPENINT>,TotalEqualWeightPr <OPENINT>.1,TotalEqualWeightPr <OPENINT>.2
1505,20210522,256514.0,256514.0,253844.0,253844.0,3140846699,1.542891e+13,171043,4.995992e+10
1506,20210523,253187.0,253187.0,251084.0,251084.0,2736872962,1.429308e+13,195077,4.941684e+10
1507,20210524,250672.0,250672.0,247768.0,247768.0,3100360796,1.610564e+13,252943,4.876418e+10
1508,20210525,247551.0,248004.0,247495.0,247495.0,3828842175,2.090582e+13,349959,4.871054e+10
1509,20210526,247514.0,247861.0,247608.0,247608.0,3187572389,1.646125e+13,389162,4.873278e+10
...,...,...,...,...,...,...,...,...,...
1739,20220516,276266.0,278289.0,278220.0,278220.0,10148885600,5.205224e+13,798234,6.713157e+10
1740,20220517,278671.0,278943.0,277344.0,277344.0,10239274235,5.180431e+13,854852,6.692013e+10
1741,20220518,277249.0,277249.0,274074.0,274074.0,8283025474,4.102303e+13,933043,6.613119e+10
1742,20220521,273924.0,273924.0,272353.0,272353.0,8136842521,4.053541e+13,590090,6.588863e+10


##############


/content/IRX6XTPI0009.csv
Overall Index
شاخص_كل6


,<DTYYYYMMDD>,Overall Index <OPEN>,Overall Index <HIGH>,Overall Index <LOW>,Overall Index <CLOSE>,Overall Index <VOL>,Overall Index <OPENINT>,Overall Index <OPENINT>.1,Overall Index <OPENINT>.2
3004,20210522,1149260.0,1149260.0,1128164.1,1128164.1,3140846699,1.542891e+13,171043,4.514616e+10
3005,20210523,1123500.0,1123500.0,1107900.3,1107900.3,2736872962,1.429308e+13,195077,4.429764e+10
3006,20210524,1105220.0,1105220.0,1095726.2,1095726.2,3100360796,1.610564e+13,252943,4.379279e+10
3007,20210525,1095800.0,1105850.0,1105446.0,1105446.0,3828842175,2.090582e+13,349959,4.413669e+10
3008,20210526,1106380.0,1110560.0,1109369.6,1109369.6,3187572389,1.646125e+13,389162,4.429966e+10
...,...,...,...,...,...,...,...,...,...
3238,20220516,1575270.0,1583800.0,1583800.5,1583800.5,10148885600,5.205224e+13,798234,6.297415e+10
3239,20220517,1588770.0,1607220.0,1606024.3,1606024.3,10239274235,5.180431e+13,854852,6.385780e+10
3240,20220518,1605300.0,1605300.0,1594977.7,1594977.7,8283025474,4.102303e+13,933043,6.341822e+10
3241,20220521,1591530.0,1591530.0,1579691.1,1579691.1,8136842521,4.053541e+13,590090,6.279521e+10


##############




## Calculate Class

In [ ]:
merged_dataset["Class"] = np.sign((merged_dataset["Overall Index <CLOSE>"] - merged_dataset["Overall Index <CLOSE>"].shift(+1)))
merged_dataset = merged_dataset.iloc[1:,:].reset_index(drop=True)
merged_dataset["Class"] = merged_dataset["Class"].apply(lambda x: int((x+1)/2))

In [ ]:
merged_dataset.isnull().any(axis=1).values.any()

False

In [ ]:
merged_dataset

,<DTYYYYMMDD>,OTC Overall Index <OPEN>,OTC Overall Index <HIGH>,OTC Overall Index <LOW>,OTC Overall Index <CLOSE>,OTC Overall Index <VOL>,OTC Overall Index <OPENINT>,OTC Overall Index <OPENINT>.1,OTC Overall Index <OPENINT>.2,Industy Index <OPEN>,...,TotalEqualWeightPr <OPENINT>.2,Overall Index <OPEN>,Overall Index <HIGH>,Overall Index <LOW>,Overall Index <CLOSE>,Overall Index <VOL>,Overall Index <OPENINT>,Overall Index <OPENINT>.1,Overall Index <OPENINT>.2,Class
0,20210523,17642.9,17642.9,17503.9,17503.9,598258196,3.181944e+12,98344,1.169097e+10,968432.0,...,4.941684e+10,1123500.0,1123500.0,1107900.3,1107900.3,2736872962,1.429308e+13,195077,4.429764e+10,0
1,20210524,17499.8,17504.5,17476.5,17476.5,529694599,2.683259e+12,115169,1.165787e+10,953029.0,...,4.876418e+10,1105220.0,1105220.0,1095726.2,1095726.2,3100360796,1.610564e+13,252943,4.379279e+10,0
2,20210525,17474.0,17552.8,17552.8,17552.8,605556635,3.236629e+12,205382,1.170879e+10,953288.0,...,4.871054e+10,1095800.0,1105850.0,1105446.0,1105446.0,3828842175,2.090582e+13,349959,4.413669e+10,1
3,20210526,17553.4,17582.5,17582.3,17582.3,528637682,2.898542e+12,233866,1.172845e+10,963015.0,...,4.873278e+10,1106380.0,1110560.0,1109369.6,1109369.6,3187572389,1.646125e+13,389162,4.429966e+10,1
4,20210529,17597.8,17732.3,17732.3,17732.3,878277335,6.110854e+12,245412,1.182190e+10,978584.0,...,4.914575e+10,1111520.0,1139620.0,1139480.9,1139480.9,5193610223,2.851750e+13,395488,4.558550e+10,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
232,20220516,21553.8,21620.7,21620.7,21620.7,2465155499,1.561998e+13,345395,1.190409e+10,1469370.0,...,6.713157e+10,1575270.0,1583800.0,1583800.5,1583800.5,10148885600,5.205224e+13,798234,6.297415e+10,1
233,20220517,21644.3,21695.6,21689.7,21689.7,2629661250,1.531904e+13,303832,1.200937e+10,1483100.0,...,6.692013e+10,1588770.0,1607220.0,1606024.3,1606024.3,10239274235,5.180431e+13,854852,6.385780e+10,1
234,20220518,21620.7,21671.6,21643.8,21643.8,1843016966,1.139007e+13,262424,1.197906e+10,1494950.0,...,6.613119e+10,1605300.0,1605300.0,1594977.7,1594977.7,8283025474,4.102303e+13,933043,6.341822e+10,0
235,20220521,21612.6,21612.6,21539.4,21539.4,1422075970,8.965528e+12,182173,1.192066e+10,1478060.0,...,6.588863e+10,1591530.0,1591530.0,1579691.1,1579691.1,8136842521,4.053541e+13,590090,6.279521e+10,0


## Convert to Time Series

In [ ]:
def create_ts_dataset(number_days):
  
  dataset_array = []
  for i in range(number_days, len(merged_dataset)):
    day = int(merged_dataset.iloc[i]["<DTYYYYMMDD>"])
    class_ = int(merged_dataset.iloc[i]["Class"])
    data = merged_dataset.drop(columns=["<DTYYYYMMDD>"])\
          .iloc[i-number_days:i].values

    dataset_array.append({"Day": day, "Data": data, "Class": class_})

  return pd.DataFrame(dataset_array)

In [ ]:
ts_dataset = create_ts_dataset(number_days=10)

## Shuffle and Split

In [ ]:
ts_dataset = ts_dataset.sample(frac=1, random_state=0)

In [ ]:
train_valid_dataset, test_dataset = train_test_split(ts_dataset, test_size=0.2)
train_dataset, valid_dataset = train_test_split(train_valid_dataset, test_size=0.125) 

In [ ]:
print(sum(train_dataset["Class"].values) * 100 / len(train_dataset["Class"].values))
print(sum(valid_dataset["Class"].values) * 100 / len(valid_dataset["Class"].values))
print(sum(test_dataset["Class"].values) * 100 / len(test_dataset["Class"].values))

55.063291139240505
69.56521739130434
58.69565217391305


# Model

In [ ]:
class MyRNN(keras.Model):
  def __init__(self, cell, number_days, number_cells, number_layers, dropout_rate):
    super().__init__()

    self.model = Sequential([
                             Input((number_days, 129)),
                             BatchNormalization(),
    ])

    for _ in range(number_layers-1):
      self.model.add(cell(number_cells, return_sequences=True))
    self.model.add(cell(number_cells))
    
    self.model.add(Dropout(dropout_rate))
    self.model.add(Dense(2, activation="softmax"))

  def call(self, inputs):
    return self.model(inputs)

# Results

## one Layer

In [ ]:
number_trial = 5

for cell in [LSTM, GRU]:
  for number_cells in [8, 32, 64, 256, 512]:
    for dropout_rate in [0, 0.5, 0.8]:
      for optimizer in [keras.optimizers.Adam(), keras.optimizers.SGD()]:
        accuracy_train_total = 0 
        accuracy_valid_total = 0
        accuracy_test_total = 0

        print(f"cell = {cell} | number_cells = {number_cells} | dropout_rate = {dropout_rate} | optimizer = {optimizer}")
        for i in range(number_trial):
          model = MyRNN(number_days=10, number_cells=number_cells, cell=cell,
                        number_layers=1, dropout_rate=dropout_rate)
          model.compile(optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=["accuracy"])
          es_callback = EarlyStopping(monitor="val_loss", patience=20, restore_best_weights=True)

          model.fit(x=np.stack(train_dataset["Data"].values),
                    y=train_dataset["Class"].values,
                    validation_data=[np.stack(valid_dataset["Data"].values),
                                    valid_dataset["Class"].values],
                    epochs=300,
                    callbacks=[es_callback],
                    batch_size=8,
                    verbose=0)

          accuracy_train = model.evaluate(x=np.stack(train_dataset["Data"].values), y=train_dataset["Class"].values, verbose=0)[1]
          accuracy_valid = model.evaluate(x=np.stack(valid_dataset["Data"].values), y=valid_dataset["Class"].values, verbose=0)[1]
          accuracy_test = model.evaluate(x=np.stack(test_dataset["Data"].values), y=test_dataset["Class"].values, verbose=0)[1]

          accuracy_train_total += accuracy_train 
          accuracy_valid_total += accuracy_valid
          accuracy_test_total += accuracy_test

        print(f"Accuracy Train = {round(accuracy_train_total/number_trial*100)}%", end=" ")
        print(f"Accuracy Valid = {round(accuracy_valid_total/number_trial*100)}%", end=" ")
        print(f"Accuracy Test = {round(accuracy_test_total/number_trial*100)}%\n")

cell = <class 'keras.layers.recurrent_v2.LSTM'> | number_cells = 8 | dropout_rate = 0 | optimizer = <keras.optimizer_v2.adam.Adam object at 0x7fad4f4ece50>
Accuracy Train = 73% Accuracy Valid = 69% Accuracy Test = 63%

cell = <class 'keras.layers.recurrent_v2.LSTM'> | number_cells = 8 | dropout_rate = 0 | optimizer = <keras.optimizer_v2.gradient_descent.SGD object at 0x7fad4f4ecd10>
Accuracy Train = 56% Accuracy Valid = 70% Accuracy Test = 54%

cell = <class 'keras.layers.recurrent_v2.LSTM'> | number_cells = 8 | dropout_rate = 0.5 | optimizer = <keras.optimizer_v2.adam.Adam object at 0x7fad4f4ec350>
Accuracy Train = 69% Accuracy Valid = 70% Accuracy Test = 60%

cell = <class 'keras.layers.recurrent_v2.LSTM'> | number_cells = 8 | dropout_rate = 0.5 | optimizer = <keras.optimizer_v2.gradient_descent.SGD object at 0x7fad4f4ece50>
Accuracy Train = 62% Accuracy Valid = 70% Accuracy Test = 64%

cell = <class 'keras.layers.recurrent_v2.LSTM'> | number_cells = 8 | dropout_rate = 0.8 | optimize

## Stacked